# import libraries

In [1]:
import requests
from scrapy.selector import Selector
from tqdm import tqdm

# Get page code source

In [2]:
page = requests.get("https://www.moroccoworldnews.com/news-2/")

# Get pages Urls

In [3]:
num_pages = Selector(text = page.text).css("div.page-nav.td-pb-padding-side a.last::text").get()
num_pages = int(num_pages.replace(",",""))

pages_urls = ["https://www.moroccoworldnews.com/news-2/page/"+str(i+1) for i in range(num_pages)]
len(pages_urls), pages_urls[-1]

(1738, 'https://www.moroccoworldnews.com/news-2/page/1738')

# Get relevant Data

In [4]:
def get_news_from_page(page_url):
    page1 = requests.get(page_url)
    titles = Selector(text=page1.text).css("div.td-block-span6 h3.entry-title.td-module-title a::text").getall()
    urls = Selector(text=page1.text).css("div.td-block-span6 h3.entry-title.td-module-title a::attr(href)").getall()
    images = Selector(text=page1.text).css("div.td-block-span6 div.td-module-image div a img::attr(src)").getall()
    authors = Selector(text=page1.text).css("div.td-block-span6 div.meta-info span.td-post-author-name a::text").getall()
    dates = Selector(text=page1.text).css("div.td-block-span6 div.meta-info span.td-post-date time::text").getall()
    headlines = [Selector(text=requests.get(url).text).css("div.td-pb-row div.first-para p::text").get() for url in urls]
    news = [
        {"title":titles[i], "author":authors[i], "date":dates[i], "headline": headlines[i], "image":images[i], "url":urls[i]} 
        for i in range(len(urls))
    ]
    return news

In [5]:
news = []
for page_url in tqdm(pages_urls[:100]):
    news.extend(get_news_from_page(page_url))
    
len(news)

100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [1:00:48<00:00, 36.49s/it]


2500

# Put data into CSV file

In [10]:
import csv 
with open('moroccoworldnews.csv', 'a', newline='\n', encoding="utf-8") as csvfile:
    fieldnames = ['title', 'author', 'date', 'headline', 'image', 'url']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for new in news: writer.writerow(new)